<p style="font-family: Arial; font-size:3.75em;color:purple; font-style:bold"><br>
LIGHT TABLE - GENERATION</p><br>

This notebook generate Light Tables based on nexus full-simulations of light, generating a DataFrame that should be used by DetSim.

TAREAS POR IMPLEMENTAR:

* Descripcion de lo que hace cada una de las funciones y sus parametros
* Anyadir Tablas de Configuracion a la salida
* Leer el setup de un fichero de configuracion
* Anyadir tests
* Posibilidad de muestrear los volumenes / areas con un patron asimetrico (mas denso a radios altos, para reflejar mejor los cambios que se producen cerca de los bordes).
* Usar ThreeVectors de IC para el pitch y posiciones ??
* Aprender a fraccionar el envío de jobs a las colas, porque un job por punto son demasiados.
* Hacer el setting de IC y de NEXUS desde las funciones, no desde el terminal
* Anyadir la opcion de correr n numero de eventos por fichero y así correr más eventos por fichero, y menos fotones por evento de manera que necesitemos menos memoria.
* Anyadir tests de que las simulaciones han corrido bien en las colas.

In [1]:
from IPython.core.display import HTML
css = open('css/style-table.css').read() + open('css/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [2]:
# General importings
import os
import numpy  as np
import pandas as pd

from math import ceil

# Specific IC stuff
import invisible_cities.core.system_of_units  as units
from invisible_cities.io.mcinfo_io        import load_mcsensor_response_df
from invisible_cities.io.mcinfo_io        import get_sensor_types


# Light Table stuff
from sim_functions   import make_init_file
from sim_functions   import make_config_file
from sim_functions   import get_num_photons
from sim_functions   import run_sim

from table_functions import get_table_positions
from table_functions import get_working_paths


# SETUP DATA

In [3]:
VERBOSITY = True

In [4]:
MAX_PHOTONS_PER_EVT = 1000000

In [9]:
# Current options: "NEXT_NEW", "NEXT100", "NEXT_FLEX"
det_name = "NEXT_NEW"

# Type of Light Table: S1 or S2
table_type = "S2"

# Detector Type (it will be PmtR11410) for most of the geometries
# but it could be "FIBER_SENSOR" for the flexible geometry
sns_type = "PmtR11410"

# Table pitch
pitch = (200.0 * units.mm, 200.0 * units.mm, 200.0 * units.mm)
#pitch = (20.0 * units.mm, 20.0 * units.mm, 40.0 * units.mm)

# Table num photons / point
photons_per_point = 100000

# Verbosity
if VERBOSITY:
    print(f"*****  {det_name} - {table_type} - {sns_type}  Light Table  *****")
    print(f"* pitch: {pitch} mm")

*****  NEXT_NEW - S2 - PmtR11410  Light Table  *****
* pitch: (200.0, 200.0, 200.0) mm


In [10]:
# Getting (num_evts / file) & num_evts
photons_per_event = photons_per_point
num_evts = ceil(photons_per_point / MAX_PHOTONS_PER_EVT)

if num_evts > 1:
    photons_per_event = MAX_PHOTONS_PER_EVT

# Verbosity
if VERBOSITY:
    print(f"* Photons/point: {photons_per_point:.1e}  ->  ")
    print(f"  Num Events: {num_evts} of {photons_per_event:.0e} photons/event")
    

* Photons/point: 1.0e+05  ->  
  Num Events: 1 of 1e+05 photons/event


In [11]:
table_positions = get_table_positions(det_name, table_type, pitch)
#table_positions = [(0,0,100), (0,0,200)]

# Vervosity
if VERBOSITY:
    print(f"* {det_name} - {table_type} table -> {len(table_positions)} points.")
    print(table_positions)

* NEXT_NEW - S2 table -> 3 points.
[(-8, -8, 0), (-8, 192, 0), (192, -8, 0)]


In [12]:
# Getting PATHS
config_path, log_path, dst_path, table_path = get_working_paths(det_name)

# Verbosity
if VERBOSITY:
    print(f"* Config PATH: {config_path}")
    print(f"* Log    PATH: {log_path}")
    print(f"* Dst    PATH: {dst_path}")
    print(f"* Table  PATH: {table_path}")

* Config PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/config/
* Log    PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/log/
* Dst    PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/dst/
* Table  PATH: /Users/Javi/Development/NextLightTable/data/NEXT_NEW/table/


# LIGHT SIMULATIONS

In [13]:
for pos in table_positions:
    
    # file names
    base_fname = f"{det_name}.x_{pos[0]}.y_{pos[1]}.z_{pos[2]}"

    init_fname   = config_path + base_fname + ".init"
    config_fname = config_path + base_fname + ".config"
    log_fname    = log_path    + base_fname + ".log"
    dst_fname    = dst_path    + base_fname + ".next"
    
    # make configuration files
    make_init_file(det_name, init_fname, config_fname)
    
    make_config_file(det_name, config_fname, dst_fname,
                     pos[0], pos[1], pos[2],
                     photons_per_event)
    
    # Runing the simulation
    if VERBOSITY:
        print(f"* Runing {det_name} sim of {photons_per_point:.1e} photons from {pos} ...")
        
    # Check if the sim is already run with the correct num_photons
    if os.path.isfile(dst_fname + '.h5'):
        run_photons = get_num_photons(dst_fname + '.h5')
        if (get_num_photons(dst_fname + '.h5') < photons_per_point):
            print("  Simulation already run previously with less events, so re-running ...")
            run_sim(init_fname, log_fname, num_evts)
        else:
            print("  Simulation already run previously, so skipping ...")
    
    else:
        run_sim(init_fname, log_fname, num_evts)

* Runing NEXT_NEW sim of 1.0e+05 photons from (-8, -8, 0) ...
  Simulation already run previously with less events, so re-running ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (-8, 192, 0) ...
* Runing NEXT_NEW sim of 1.0e+05 photons from (192, -8, 0) ...


# TABLE GENERATION

Initial list to be filled with data.

It is the seed of the Light Table DataFrame.

In [14]:
# Getting the list of sensors from the first dst file
pos          = table_positions[0]

base_fname   = f"{det_name}.x_{pos[0]}.y_{pos[1]}.z_{pos[2]}"
dst_fname    = dst_path + base_fname + ".next.h5"
sensor_types = get_sensor_types(dst_fname)

# Add check that the number of sensors is the expected one

In [15]:
sensor_ids = sensor_types[sensor_types.sensor_name == sns_type].sensor_id.tolist()
sensor_ids.sort()

In [16]:
light_table_columns = np.append(['x', 'y', 'z'], np.append(sensor_ids, 'total'))
light_table_data    = []

In [17]:
# Getting the table data from sims
for pos in table_positions:
    
    # Verbosity
    if VERBOSITY:
        print(f"\n* Getting data from {det_name} - {pos} ...")

    # Getting the right DST file name
    base_fname = f"{det_name}.x_{pos[0]}.y_{pos[1]}.z_{pos[2]}"
    dst_fname  = dst_path + base_fname + ".next.h5"
    
    # Add a check that the position is the expected one ??
    # Add check that the number of sensors is the expected one
    if os.path.isfile(dst_fname):
    
        # Getting the number of photons from the file, as it could be different
        # from the one included in the setup.
        num_photons = get_num_photons(dst_fname)
        
        # Getting the sensor response of the sns_type requested
        sns_response = load_mcsensor_response_df(dst_fname, sns_name = sns_type)
        sns_charge = sns_response.groupby('sensor_id').charge.sum().tolist()
        sns_charge   = np.divide(sns_charge, num_photons)
        sns_charge   = np.append(sns_charge, sum(sns_charge))
    
        # Composing & storing this position data
        pos_data = np.append([pos[0], pos[1], pos[2]], sns_charge)
        light_table_data.append(pos_data)
        
        # Verbosity
        if VERBOSITY:
            print(f"  Simulation run with {num_photons:9} initial photons.")
            #print(f"  {sns_type} behaviour: {sns_response.charge.describe()}")

    # If the DST file does NOT EXIST
    else:
        print(f"  WARNING: {dst_fname} NOT exist.")



* Getting data from NEXT_NEW - (-8, -8, 0) ...
  Simulation run with    100000 initial photons.

* Getting data from NEXT_NEW - (-8, 192, 0) ...
  Simulation run with    100000 initial photons.

* Getting data from NEXT_NEW - (192, -8, 0) ...
  Simulation run with    100000 initial photons.


In [22]:
### Building the LightTable DataFrame
light_table = pd.DataFrame(light_table_data,
                          columns = light_table_columns)

# Setting indices
if(table_type == 'S1'):
    light_table.set_index(['x', 'y', 'z'], inplace = True)
else:
    light_table.drop(columns='z', inplace = True)
    light_table.set_index(['x', 'y'], inplace = True)

light_table.sort_index() # Not sure if needed to speed access

light_table = light_table.rename(columns = lambda name: sns_type + '_' + name)

light_table.head()

PmtR11410_0  PmtR11410_1  PmtR11410_2  PmtR11410_3  \
x      y                                                            
-8.0   -8.0        0.00033      0.00040      0.00034      0.00017   
        192.0      0.00025      0.00028      0.00028      0.00021   
 192.0 -8.0        0.00019      0.00025      0.00020      0.00025   

               PmtR11410_4  PmtR11410_5  PmtR11410_6  PmtR11410_7  \
x      y                                                            
-8.0   -8.0        0.00022      0.00013      0.00019      0.00015   
        192.0      0.00021      0.00016      0.00032      0.00014   
 192.0 -8.0        0.00011      0.00011      0.00016      0.00014   

               PmtR11410_8  PmtR11410_9  PmtR11410_10  PmtR11410_11  \
x      y                                                              
-8.0   -8.0        0.00021      0.00021       0.00014       0.00022   
        192.0      0.00012      0.00024       0.00017       0.00017   
 192.0 -8.0        0.00027      0.00019       0.00018       0.00025   

               PmtR11410_total  
x      y                        
-8.0   -8.0            0.00271  
        192.0          0.00255  
 192.0 -8.0            0.00230

In [23]:
# Storing the DataFrame

light_table_fname = table_path + f"{det_name}.{table_type}.{sns_type}.LightTable.h5"

if VERBOSITY:
    print(f"\n*** Storing Light Table in {light_table_fname} ...")

light_table.to_hdf(light_table_fname, '/LightTable',
                   mode   = 'w',
                   format = 'table',
                   data_columns = True)


*** Storing Light Table in /Users/Javi/Development/NextLightTable/data/NEXT_NEW/table/NEXT_NEW.S2.PmtR11410.LightTable.h5 ...
